In [1]:
import pandas as pd
import numpy as np
path = 'Donor2020.xlsx'
df = pd.read_excel(open(path, 'rb'))

In [2]:
df.shape

(40814, 83)

In [3]:
df_tag_list = df.tag_list.str.split(',')

In [4]:
df_tag_colon = df_tag_list.apply(lambda x: [c.strip() for c in x if ':' in c])
df_tag_colon

0                                     [CASS: Standardized]
1        [CASS: Standardized, Income: $75000 - $99999, ...
2        [CASS: Standardized, EstWealth: Greater than $...
3                                     [CASS: Standardized]
4        [CASS: Standardized, MedHomeValue: $350000 - $...
                               ...                        
40809                                 [CASS: Standardized]
40810    [CASS: Standardized, EstWealth: $250000 - $499...
40811    [CASS: Standardized, EstWealth: Greater than $...
40812    [CASS: Standardized, EstWealth: Greater than $...
40813                                 [CASS: Standardized]
Name: tag_list, Length: 40814, dtype: object

In [5]:
df_tag_rest = df_tag_list.apply(lambda x: [c.strip().lower() for c in x if ':' not in c])
df_tag_rest

0                                                       []
1        [donor, librarian, loc, petition, carla hayden...
2        [school libraries petition, school library sup...
3        [rsvp, donor, one time donor, alamw16, fundrai...
4        [librarian, petition, dogooder, federal, child...
                               ...                        
40809     [petition, email to politicians, dogooder, nses]
40810    [donor, fundraiser, librarian, tickets purchas...
40811    [petition, dogooder, njla 2019, childrenpresen...
40812    [petition, dogooder, njla 2019, childrenpresen...
40813    [librarian, john's list, iowa 2019 training, n...
Name: tag_list, Length: 40814, dtype: object

In [6]:
import itertools
tags_with_colon = set(itertools.chain.from_iterable(df_tag_colon))

In [7]:
info_tags = ['EstWealth', 'Ethnicity', 'Income', 'Residence', 'School', 'MedHomeValue']

In [8]:
def get_column(l, info):
    l_info = [c for c in l if info in c]
    if len(l_info) > 0:
        return l_info[0].split(':')[1].strip()
    return ''
        
for info in info_tags:
    df[info] = df_tag_colon.apply(get_column, args=(info,))

In [9]:
df_X = df[info_tags].copy()

In [10]:
def to_num(x):
    num = ''.join([c for c in list(x.split('-')[0]) if c.isdigit()])
    try:
        return float(num)
    except:
        return np.nan
    
to_num_list = ['EstWealth', 'Income', 'MedHomeValue']
for col in to_num_list:
    df_X[col] = df_X[col].apply(to_num)

In [11]:
y = df.is_donor.values
y

array([False,  True, False, ..., False, False, False])

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label_list = ['Ethnicity', 'Residence']
for col in label_list:
    df_X[col] = le.fit_transform(df_X[col])

In [13]:
df_X.School.value_counts()

                                 27430
Completed College                 5584
Completed High School             4306
Completed Graduate School         3336
Attended Vocational/Technical      158
Name: School, dtype: int64

In [14]:
school_dict = {'Completed College': 3,
               'Completed High School': 2, 
               'Completed Graduate School': 4,
               'Attended Vocational/Technical': 1,
               '': None}
df_X.School = df_X.School.apply(lambda x: school_dict[x])

In [15]:
X = df_X.values
Xy = np.hstack([X, y.reshape(-1,1)])

In [33]:
df_Xy = pd.DataFrame(Xy, columns=info_tags + ['y'])
df_Xy.corr()

,EstWealth,Ethnicity,Income,Residence,School,MedHomeValue,y
EstWealth,1.000000,0.066626,0.626236,0.072770,0.224833,0.601388,0.002226
Ethnicity,0.066626,1.000000,0.060026,0.537630,0.027739,0.053749,0.003599
Income,0.626236,0.060026,1.000000,0.077633,0.196115,0.571999,0.019455
Residence,0.072770,0.537630,0.077633,1.000000,0.008408,0.042049,-0.015958
School,0.224833,0.027739,0.196115,0.008408,1.000000,0.179481,0.011749
MedHomeValue,0.601388,0.053749,0.571999,0.042049,0.179481,1.000000,0.042471
y,0.002226,0.003599,0.019455,-0.015958,0.011749,0.042471,1.000000


In [40]:
df_Xy_clean = df_Xy.dropna()
df_Xy_clean.corr()

,EstWealth,Ethnicity,Income,Residence,School,MedHomeValue,y
EstWealth,1.000000,0.084558,0.591233,0.032827,0.214264,0.605796,0.014624
Ethnicity,0.084558,1.000000,0.067186,-0.004538,0.030996,0.055376,0.039811
Income,0.591233,0.067186,1.000000,0.036087,0.185278,0.593855,0.039351
Residence,0.032827,-0.004538,0.036087,1.000000,0.007255,0.026157,0.022450
School,0.214264,0.030996,0.185278,0.007255,1.000000,0.178859,0.010649
MedHomeValue,0.605796,0.055376,0.593855,0.026157,0.178859,1.000000,0.032508
y,0.014624,0.039811,0.039351,0.022450,0.010649,0.032508,1.000000


In [41]:
X = df_Xy_clean.drop('y', axis=1).values
y = df_Xy_clean.y.values

In [42]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X, y)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
y_pred = lr.predict(X)

In [48]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y, y_pred)

In [49]:
acc

0.9547558738754477

In [50]:
from sklearn.model_selection import cross_val_score

In [53]:
cross_val_score(lr, X, y, cv=5)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

array([0.95460498, 0.95458515, 0.95458515, 0.95500218, 0.95500218])